In [1]:
from morpheus.project.types.Permissions import Role
from morpheus.project.types.User import UserId

user_id = UserId.new()

members: dict[UserId, Role] = {
    user_id: Role.ADMIN
}

In [2]:
if user_id in members:
    print("User is a member")
else:
    print("User is not a member")

User is a member


In [3]:
from morpheus.project.types.Permissions import Permissions

owner_id = UserId.new()
permissions = Permissions.new(owner_id)

permissions.member_can_edit(user_id)

False